#### Launching JupyterLab
Click on <b> Interactive Apps </b>, then <b>Jupyter</b>. </br>
![interactive_session.png](imgs/interactive_session.png) </br>
Specify the following input values on the page, replacing "Account Name" with your project group name. You may also wish to change the number of hours based on how long you intend to work on this tutorial for now. </br>
- Python Version: 3.10.8
- Lab or Notebook: JupyterLab
- Working Directory: <b> path to desired project directory </b>
- Account Name: geospatialworkshop
- Partition Name: atlas
- QOS: ood – Max Time: 8-00:00:00
- Number of hours: 4
- Number of nodes: 1
- Number of tasks: 1
- Additional Slurm Parameters: --mem=32gb

Then click the `Launch` button at the bottom of the page. Once your session loads, click the `Connect to Jupyter` button.

Once the jupyter session is launched, we will open up a terminal. Click the `+` button on the top right, above the navigation pane.</br>
![plus_button.png](imgs/plus_button.png)</br>
Then click on the `Terminal` button. </br>
![open_terminal.png](imgs/open_terminal.png) </br>

<a id='var_setup'></a>
#### Setting Project Shell Variables
In the terminal, use the following commands to save your project group name and project directory path as shell variables. If you are not a part of the "geospatialworkshop" project group, replace "geospatialworkshop" in the first line with the name of a project group that you are a part of. 

Next, decide on a project directory location. You may use your home directory, though you may quickly run out of space, so we recommend using the 90daydata directory instead. If you have space in a project directory that you would prefer to use over 90daydata, modify the path in the second command. Otherwise, leave the command as is to use 90daydata. 

Navigate to the directory you would like to store your project directory in, and run the following commands. This will create your project directory, store the project directory path into the shell variable `project_dir`, and to store your project group name into the shell variable `project_name`. If you are not a part of the geospatialworkshop group, replace "geospatialworkshop" with the name of a project group that you are a part of. </br></br>
`mkdir rv_workbook` </br>
``project_dir=`pwd`/rastervision`` </br>
`project_name=geospatialworkshop` </br>

#### Transferring Workshop Files to Project Directory
This workshop refers to files stored in the `/reference/workshops/rastervision` folder. We will only transfer some of the contents of `/reference/workshops/rastervision` to our project directory because some of the files are very large and can be referenced in-place.

Use the following commands to copy the reference files to your project directory. </br>
`cd $project_dir` </br>
`cp /reference/workshops/rastervision/model/ .` </br>
`cp /reference/workshops/rastervision/Raster_Vision_workbook.ipynb .` </br>

#### Creating the Kernel

NOA: Update this to refer to a kernel in /reference/workshops/rastervision
(First test in 90daydata, then ask for a transfer)

Run these commands in the terminal to create the jupyter kernel: </br>
`source /project/geospatialworkshop/workshop_venv/bin/activate` </br>
`ipython kernel install --name "grwg_workshop" --user` </br>
`cp /project/geospatialworkshop/grwg_workshop.json ~/.local/share/jupyter/kernels/grwg_workshop/kernel.json` </br>

#### Open Workbook

From the navigation pane on the left side of the screen, navigate to your `rv_workbook` directory.</br>
</br>
![open_workbook_directory.png](imgs/open_workbook_directory.png) </br>
Next, click on `Raster_Vision_workbook.ipynb` to launch the workbook.</br>
![open_workbook.png](imgs/open_workbook.png) </br>

Lastly, set the kernel by clicking on the `Kernel` tab, selecting `Change Kernel...`, and then selecting the `grwg_workshop` kernel. </br>
![change_kernel.png](imgs/change_kernel.png)

# Semantic Segmentation of Aerial Imagery with Raster Vision 
## Part 2: Overview of Deep Learning for Imagery and the Raster Vision Pipeline

This tutorial series walks through an example of using [Raster Vision](https://rastervision.io/) to train a deep learning model to identify buildings in satellite imagery.</br>

*Primary Libraries and Tools*:

|Name|Description|Link|
|-|-|-|
| `Raster Vision ` | Library and framework for geospatial semantic segmentation, object detection, and chip classification with python| https://rastervision.io/ |
| `Singularity` | Containerization software that allows for transportable and reproducible software | https://docs.sylabs.io/guides/3.5/user-guide/introduction.html |
| `pandas` | Dataframes and other datatypes for data analysis and manipulation | https://pandas.pydata.org/ |
| `geopandas` | Extends datatypes used by pandas to allow spatial operations on geometric types | https://geopandas.org/en/stable/ |
| `rioxarray` | Data structures and routines for working with gridded geospatial data | https://github.com/corteva/rioxarray |
| `plotnine` | A plotting library for Python modeled after R's [ggplot2](https://ggplot2.tidyverse.org/) | https://plotnine.readthedocs.io/en/v0.12.3/ |
| `pathlib` | A Python library for handling files and paths in the filesystem | https://docs.python.org/3/library/pathlib.html |

*Prerequisites*:
  * Basic understanding of navigating the Linux command line, including navigating among directories and editing text files
  * Basic python skills, including an understanding of object-oriented programming, function calls, and basic data types
  * Basic understanding of shell scripts and job scheduling with SLURM for running code on Atlas
  * A SCINet account for running this tutorial on Atlas
  * **Completion of tutorial part 1 of this series**

*Tutorials in this Series*:
  * 1\. **Tutorial Setup on SCINet**
  * 2\. **Overview of Deep Learning for Imagery and the Raster Vision Pipeline <span style="color: red;">_(You are here)_</span>**
  * 3\. **Constructing and Exploring the Singularity Image**
  * 4\. **Exploring the dataset and problem space**
  * 5\. **Overview of Raster Vision Model Configuration and Setup**
  * 6\. **Breakdown of Raster Vision Code Version 1**
  * 7\. **Evaluating training performance and visualizing predictions**

# 1. Overview of Deep Learning for Imagery Concepts

#### What is a Neural Network
A neural network is essentially a complicated mathematical function that receives inputs, such as images, and outputs predictions, such as image classification. A neural network has very many, often millions of parameters that control its functionality. You can think of each parameter as a dial, and the process of training a model involves adjusting the parameters to improve the model's performance. The process of model training involves passing data through the model, observing the model's accuracy, applying slight adjustments to the parameters to improve model performance, and repeating. If you are interested in learning more about the inner workings of neural networks, you can find more information here (NOA FIND A GOOD RESOURCE). For this tutorial, we do not need an in depth understanding of the inner workings of a neural network, since we are not building and training a neural network from scratch. Rather, this tutorial will focus on how to use the Raster Vision framework to set up model training for a geospatial deep learning task with transfer learning. </br>

#### Process of training a neural network:
- Acquire a fully-labeled dataset.
- Split dataset into training, validating, and testing sets.
- Define model structure (or select pre-trained model if using transfer learning).
- Training loop:
    - Split the training dataset into batches.
    - For each batch of data:
         - Pass the batch of data through the model.
         - Observe the model accuracy.
         - Update the model parameters to improve model performance.
- Iterate through the training loop several times.
- Run the model on the validation data and observe performance. This allows you to guage how well the model performs on data it has not been trained on. Modify training procedure as desired, and train again.
- Once you have a model that you are happy with, then run the model on the test data. This will guage how well the model performs on data is has not been trained or validated on.
- Deploy model for use.

#### What is Transfer Learning

Training a neural network from scratch requires a lot of time and computational resources because there are so many parameters to tune. Transfer Learning is a very common approach used to decrease the time it takes to train a model. With transfer learning, we first find a model that has already been trained to perform a certain task. Then, we train that model on a new task. For example, say we wish to build a model that can identify trucks in images. If we already have a model that is trained to identify cars in images, then we can use that model as the starting point of our training procedure, and further train our pre-trained model using a dataset of truck images. This will work a lot faster than building a new model from scratch. </br>

For this tutorial, we will use the [ResNet50 model](https://arxiv.org/abs/1512.03385), which is pre-trained on the [ImageNet dataset](https://www.image-net.org/index.php). The ImageNet dataset contains over a million labeled images of objects in 1000 different classes, such as "canoe", "isopod", "acorn", and "miniature schnauzer". Since the ImageNet dataset contains a large breadth of image classes, we know the ResNet50 model can extract various image features, and can thus be applied to diverse use cases.

#### Hyperparameters

<b>Parameters</b> are the "dials" within the model that are adjusted to improve the training accuracy. Parameter values are not directly set or updated by the analyst. Rather, they are initialized and updated through the model training process. <b>Hyperparameters</b>, on the other hand, are variables that control the process of training. Hyperparameters are set manually by the analyst, and analysts will often try a variety of different hyperparameter values to see which yields the best model. </br></br>
Examples of hyperparameters include:
- Number of epochs: the number of times we pass the entire training set through the model during model training.
- Batch size: the number of individual samples (ie labeled image chips) we pass through the model before updating the model parameters. Through the training process, we pass a batch of data through the model, observe the model performance, update the model parameters, and repeat. Once we have passed all of the training data through the dataset, we have completed one epoch. So, there are multiple rounds of parameter updates per epoch, and this is controlled by the batch size.
- Learning rate: a scaling factor for the magnitude of adjustments to parameters. If we have too small of a learning rate, we will take very small "steps", and training will be slow. If we have too large of a learning rate, we won't have very fine-tune control of our parameters, so we may not be able to optimize our model very well.

#### Image Chipping

In geospatial data science, we often have very large datasets from satellite or drone imagery. Neural Networks generally operate on much smaller input sizes, so instead of passing an entire satellite image through a neural network, we break up our large imagery into smaller, bite-sized pieces called "chips". Since neural networks often expect input images to all be the same size, our chips are created to have consistent pixel dimensions. Chips can be sampled from a dataset either in a grid-like fashion, or by random sampling. The chip size is chosen by the analyst to fit the problem context, and various chip sizes can be tried. </br>

Note: Some resources use the term "tile" instead of "chip".

#### Image Classification

There are many different Deep Learning tasks we may wish to perform. Image Classification is the most basic deep learning task for image-based data. The goal of Image Classification is to input an image to a model, and have the model output the image's class. For example, an Image Classification model could be trained to classify pictures as either "cats" or "dogs". Note that Image Classification models have a pre-defined set of classes to choose from, so if you have a model that can only choose between "cats" and "dogs", and you give that model a picture of a pig, the model will still return a prediction of either "cats" or "dogs".

For geospatial applications, we classify chips of our dataset instead of entire images. Hence, the Raster Vision documentation refers to this task as "Chip Classification" instead of "Image Classification" for clarity.

#### Object Detection

Object Detection allows us to find objects within images. Image Classification can tell us, for example, that a picture is of a cat. Image Classification cannot tell us where in the image the cat is, or how many cats are in the image. An Object Detection model will output bounding boxes around objects of interest. </br>
![IC vs OD](http://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1522766480/1_6j34dAOTijqP6HDFnjxPFA_udggex.png)
###### Object Detection Example from [DataCamp](https://www.datacamp.com/tutorial/object-detection-guide)</br>
Geospatial example: Object Detection could be used to analyze traffic conditions by detecting and counting cars on roads.

#### Semantic Segmentation

Semantic Segmentation models provide classification for every pixel within an image. While semantic segmentation doesn't allow us to count individual instances of objects, it does provide us with more detailed outlines of where one class ends and the next begins.</br>

![SS ex](https://assets-global.website-files.com/614c82ed388d53640613982e/63f498f8d4fe7da3b3a60cc2_semantic%20segmentation%20vs%20instance%20segmentation.jpg) 
###### Semantic Segmentation Image from [Li, Johnson, and Yeung](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture11.pdf)</br>
Geospatial example: Semantic Segmentation could be used to identify buildings in satellite images.

## 2. The Raster Vision Pipeline

##### "Raster Vision is an open source library and framework for Python developers building computer vision models on satellite, aerial, and other large imagery sets (including oblique drone imagery). There is built-in support for chip classification, object detection, and semantic segmentation using PyTorch." [(rastervision.io)](https://rastervision.io/) </br>
Raster Vision, a geospatial software tool produced by the company [Azavea](https://www.azavea.com/), can be used as either a framework or as a library. The Raster Vision framework abstracts away many technical details of geospatial deep learning, and allows users to customize and run a deep learning pipeline. Advanced python programmers can use the Raster Vision library to use pieces of Raster Vision code in their own projects. We will focus solely on how to use the Raster Vision framework in this tutorial. </br></br>
Raster Vision is built on pytorch, which is a popular python library used for building and training neural networks. The Raster Vision framework utilizes a pipeline of execution that performs a series of steps to prepare the data, train the model, use the model to predict on the validation set, calculate evaluation metrics, and bundle the model for deployment. </br>

![RV pipeline](https://docs.rastervision.io/en/0.21/_images/rv-pipeline-overview.png) 
###### [Raster Vision Pipeline Image Source](https://docs.rastervision.io/en/0.21/framework/pipelines.html)</br>

Raster Vision is a low-code platform. Users will still need to write python code to specify how they want to build their model, however they will need to write much less code than if they were building the same model from scratch in pytorch. For example, users will not have to write code to chip the data or perform the training loop, but they will need to specify the chip size, the method for constructing chips, what model architecture to use, and which of the three supported Deep Learning tasks to perform (chip classification, object detection, or semantic segmentation). </br></br>

Raster Vision is ideal for researchers who:
* Have large, fully labelled geospatial datasets they wish to expand to cover additional sites
    * Ex: satellite imagery, and associated vector data outlining objects of interest for Object Detection
    * Ex: aerial drone imagery, and associated raster data representing segmentation masks for Semantic Segmentation
* Can run their code on Atlas to take advantage of GPU acceleration
* Have python experience

##### Note: Raster Vision is not backwards compatible. When reading through documentation, ensure you are looking at the right version of Raster Vision. This tutorial is based on version 0.21.
Some older versions of documentation do not contain the most up-to-date versions of Raster Vision, and will not list version 0.21 as an option. The most up-to-date documentation can be found at [rastervision.io](https://rastervision.io/).